In [97]:
import argparse
import os
import sys
import datetime
import time
import math
import json
from pathlib import Path

import numpy as np
from PIL import Image
import torch
import torch.nn as nn
import torch.distributed as dist
import torch.backends.cudnn as cudnn
import torch.nn.functional as F
from torchvision import datasets, transforms
from torchvision import models as torchvision_models

import utils
import vision_transformer as vits
from vision_transformer import DINOHead

import easydict
import torchvision
from tqdm import tqdm

In [98]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [99]:
args = easydict.EasyDict({
     # Model parameters
    "arch" : 'vit_small',
    'patch_size' : 16,
    'out_dim' : 100,
#     'out_dim' : 65536,
    'norm_last_layer' : True,
    'momentum_teacher' : 0.996,
    'use_bn_in_head' : False,
    # Temperature teacher parameters
    'warmup_teacher_temp' : 0.04,
    'teacher_temp' : 0.04,
    'warmup_teacher_temp_epochs' : 0,
    # Training/Optimization parameters
    'use_fp16' : True,
    'weight_decay' : 0.04,
    'weight_decay_end' : 0.4,
    'clip_grad' : 3.0,
    'batch_size_per_gpu' : 64,
    'epochs' : 100,
    'freeze_last_layer' : 1,
    'lr' : 0.0005,
    'warmup_epochs' : 10,
    'min_lr' : 1e-6,
    'optimizer' : 'adamw',
    'drop_path_rate' : 0.1,
    # Multi-crop parameters
    'global_crops_scale': (0.4, 1.),
    'local_crops_number' : 8,
    'local_crops_scale' : (0.05, 0.4),
     # Misc
    'data_path' : 'C:\\Users\\dhkim\\Desktop\\directory\\imagenet_data',
    'output_dir' : 'C:\\Users\\dhkim\\Desktop\\directory\\dino-main/main_pth',
    'saveckp_freq' : 20,
    'seed' : 0,
    'num_workers' : 0,
#     'dist_url' : 'env://'
    'local_rank' : 0,
    'gpu' : device
   })

start_epoch = 0

In [100]:
utils.fix_random_seeds(args.seed)
cudnn.benchmark = True

In [101]:
class DataAugmentationDINO(object):
    def __init__(self, global_crops_scale, local_crops_scale, local_crops_number):
        flip_and_color_jitter = transforms.Compose([
            transforms.RandomHorizontalFlip(p=0.5),
            transforms.RandomApply(
                [transforms.ColorJitter(brightness=0.4, contrast=0.4, saturation=0.2, hue=0.1)],
                p=0.8
            ),
            transforms.RandomGrayscale(p=0.2),
        ])
        normalize = transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)),
        ])

        # first global crop
        self.global_transfo1 = transforms.Compose([
            transforms.RandomResizedCrop(224, scale=global_crops_scale, interpolation=Image.BICUBIC),
            flip_and_color_jitter,
            utils.GaussianBlur(1.0),
            normalize,
        ])
        # second global crop
        self.global_transfo2 = transforms.Compose([
            transforms.RandomResizedCrop(224, scale=global_crops_scale, interpolation=Image.BICUBIC),
            flip_and_color_jitter,
            utils.GaussianBlur(0.1),
            utils.Solarization(0.2),
            normalize,
        ])
        # transformation for the local small crops
        self.local_crops_number = local_crops_number
        self.local_transfo = transforms.Compose([
            transforms.RandomResizedCrop(96, scale=local_crops_scale, interpolation=Image.BICUBIC),
            flip_and_color_jitter,
            utils.GaussianBlur(p=0.5),
            normalize,
        ])

    def __call__(self, image):
        crops = []
        crops.append(self.global_transfo1(image))
        crops.append(self.global_transfo2(image))
        for _ in range(self.local_crops_number):
            crops.append(self.local_transfo(image))
        return crops

In [102]:
class DINOLoss(nn.Module):
    def __init__(self, out_dim, ncrops, warmup_teacher_temp, teacher_temp,
                 warmup_teacher_temp_epochs, nepochs, student_temp=0.1,
                 center_momentum=0.9):
        super().__init__()
        self.student_temp = student_temp
        self.center_momentum = center_momentum
        self.ncrops = ncrops
        self.register_buffer("center", torch.zeros(1, out_dim))
        # we apply a warm up for the teacher temperature because
        # a too high temperature makes the training instable at the beginning
        self.teacher_temp_schedule = np.concatenate((
            np.linspace(warmup_teacher_temp,
                        teacher_temp, warmup_teacher_temp_epochs),
            np.ones(nepochs - warmup_teacher_temp_epochs) * teacher_temp
        ))
        

    def forward(self, student_output, teacher_output, epoch, target_labels):
        """
        Cross-entropy between softmax outputs of the teacher and student networks.
        """
        criterion = nn.CrossEntropyLoss()
        
        student_out = student_output / self.student_temp
#         print("student_out :", student_out)
#         print("student_out_shape :", student_out.shape)        
        student_out = student_out.chunk(self.ncrops)
#         for i, chunk in enumerate(student_out):
#             print(f"Chunk {i} shape: {chunk.shape}")
        

        # teacher centering and sharpening
        temp = self.teacher_temp_schedule[epoch]
        teacher_out = F.softmax((teacher_output - self.center) / temp, dim=-1)
#         print("teacher_out : ", teacher_out)
#         print("teacher_out_shape : ", teacher_out.shape)
        teacher_out = teacher_out.detach().chunk(2)

        total_loss = 0
        n_loss_terms = 0
        correct_predictions = 0  # Initialize correct_predictions variable
        total_predictions = 0

        for iq, q in enumerate(teacher_out):
            for v in range(len(student_out)):
                if v == iq:
                    # we skip cases where student and teacher operate on the same view
                    continue
                loss = criterion(student_out[v], target_labels)
                total_loss += loss
                n_loss_terms += 1
        total_loss /= n_loss_terms
        self.update_center(teacher_output)
        
         # Calculate accuracy
        _, predicted_labels = torch.max(student_out[v], dim=1)
        print("predicted_labels :",predicted_labels )
        correct_predictions += (predicted_labels == target_labels).sum().item()
        total_predictions += target_labels.size(0)
        accuracy = correct_predictions / total_predictions *100
        print("accuracy :", accuracy)
        
        return total_loss, accuracy

    @torch.no_grad()
    def update_center(self, teacher_output):
        """
        Update center used for teacher output.
        """
        batch_center = torch.sum(teacher_output, dim=0, keepdim=True)
#         dist.all_reduce(batch_center)
        batch_center = batch_center / (len(teacher_output))

        # ema update
        self.center = self.center * self.center_momentum + batch_center * (1 - self.center_momentum)

In [103]:
transform = DataAugmentationDINO(
    args.global_crops_scale,
    args.local_crops_scale,
    args.local_crops_number,
)

dataset = torchvision.datasets.CIFAR100(root='../data', train=True,
                                download=True, transform=transform)

data_loader = torch.utils.data.DataLoader(
    dataset,
    batch_size=args.batch_size_per_gpu,
    num_workers=args.num_workers,
    pin_memory=True,
    drop_last=True,
)
print(f"Data loaded: there are {len(dataset)} images.")

Files already downloaded and verified
Data loaded: there are 50000 images.


In [104]:
student = vits.__dict__[args.arch](
    patch_size=args.patch_size,
    drop_path_rate=args.drop_path_rate,  # stochastic depth
)
teacher = vits.__dict__[args.arch](patch_size=args.patch_size)
embed_dim = student.embed_dim

In [105]:
student = utils.MultiCropWrapper(
    student, DINOHead(
    embed_dim,
    args.out_dim,
    use_bn=args.use_bn_in_head,
    norm_last_layer=args.norm_last_layer,
))
teacher = utils.MultiCropWrapper(
    teacher,
    DINOHead(embed_dim, args.out_dim, args.use_bn_in_head),
)
# fc_input_dim = 65536  # 이전 레이어의 출력 차원
# fc_output_dim = 100  # 원하는 출력 차원

# # 마지막 FC 레이어 추가
# student.fc = nn.Linear(fc_input_dim, fc_output_dim)
# teacher.fc = nn.Linear(fc_input_dim, fc_output_dim)



# move networks to gpu
student, teacher = student.cuda(), teacher.cuda()

In [106]:
if utils.has_batchnorms(student):
    student = nn.SyncBatchNorm.convert_sync_batchnorm(student)
    teacher = nn.SyncBatchNorm.convert_sync_batchnorm(teacher)

    # we need DDP wrapper to have synchro batch norms working...
    teacher = nn.parallel.DistributedDataParallel(teacher, device_ids=[args.gpu])
    teacher_without_ddp = teacher.module
else:
    # teacher_without_ddp and teacher are the same thing
    teacher_without_ddp = teacher
student.to(args.gpu)
# teacher and student start with the same weights
teacher_without_ddp.load_state_dict(student.state_dict())

<All keys matched successfully>

In [107]:
for p in teacher.parameters():
    p.requires_grad = False
print(f"Student and Teacher are built: they are both {args.arch} network.")

Student and Teacher are built: they are both vit_small network.


In [108]:
dino_loss = DINOLoss(
        args.out_dim,
        args.local_crops_number + 2,  # total number of crops = 2 global crops + local_crops_number
        args.warmup_teacher_temp,
        args.teacher_temp,
        args.warmup_teacher_temp_epochs,
        args.epochs,
    ).cuda()

In [109]:
params_groups = utils.get_params_groups(student)
if args.optimizer == "adamw":
    optimizer = torch.optim.AdamW(params_groups)  # to use with ViTs

In [110]:
# for mixed precision training
fp16_scaler = None
if args.use_fp16:
    fp16_scaler = torch.cuda.amp.GradScaler()

In [111]:
lr_schedule = utils.cosine_scheduler(
    args.lr * (args.batch_size_per_gpu * utils.get_world_size()) / 256.,  # linear scaling rule
    args.min_lr,
    args.epochs, len(data_loader),
    warmup_epochs=args.warmup_epochs,
)
wd_schedule = utils.cosine_scheduler(
    args.weight_decay,
    args.weight_decay_end,
    args.epochs, len(data_loader),
)
# momentum parameter is increased to 1. during training with a cosine schedule
momentum_schedule = utils.cosine_scheduler(args.momentum_teacher, 1, args.epochs, len(data_loader))
print(f"Loss, optimizer and schedulers ready.")

Loss, optimizer and schedulers ready.


In [112]:
print(student)

MultiCropWrapper(
  (backbone): VisionTransformer(
    (patch_embed): PatchEmbed(
      (proj): Conv2d(3, 384, kernel_size=(16, 16), stride=(16, 16))
    )
    (pos_drop): Dropout(p=0.0, inplace=False)
    (blocks): ModuleList(
      (0): Block(
        (norm1): LayerNorm((384,), eps=1e-06, elementwise_affine=True)
        (attn): Attention(
          (qkv): Linear(in_features=384, out_features=1152, bias=True)
          (attn_drop): Dropout(p=0.0, inplace=False)
          (proj): Linear(in_features=384, out_features=384, bias=True)
          (proj_drop): Dropout(p=0.0, inplace=False)
        )
        (drop_path): Identity()
        (norm2): LayerNorm((384,), eps=1e-06, elementwise_affine=True)
        (mlp): Mlp(
          (fc1): Linear(in_features=384, out_features=1536, bias=True)
          (act): GELU(approximate='none')
          (fc2): Linear(in_features=1536, out_features=384, bias=True)
          (drop): Dropout(p=0.0, inplace=False)
        )
      )
      (1): Block(
       

In [113]:
print(teacher)

MultiCropWrapper(
  (backbone): VisionTransformer(
    (patch_embed): PatchEmbed(
      (proj): Conv2d(3, 384, kernel_size=(16, 16), stride=(16, 16))
    )
    (pos_drop): Dropout(p=0.0, inplace=False)
    (blocks): ModuleList(
      (0): Block(
        (norm1): LayerNorm((384,), eps=1e-06, elementwise_affine=True)
        (attn): Attention(
          (qkv): Linear(in_features=384, out_features=1152, bias=True)
          (attn_drop): Dropout(p=0.0, inplace=False)
          (proj): Linear(in_features=384, out_features=384, bias=True)
          (proj_drop): Dropout(p=0.0, inplace=False)
        )
        (drop_path): Identity()
        (norm2): LayerNorm((384,), eps=1e-06, elementwise_affine=True)
        (mlp): Mlp(
          (fc1): Linear(in_features=384, out_features=1536, bias=True)
          (act): GELU(approximate='none')
          (fc2): Linear(in_features=1536, out_features=384, bias=True)
          (drop): Dropout(p=0.0, inplace=False)
        )
      )
      (1): Block(
       

In [114]:
print("Starting DINO training !")
for epoch in tqdm(range(start_epoch, args.epochs)):
    for it, (images, target_label) in tqdm(enumerate(data_loader)):         
#         print("Number of images in the list:", len(images))
        
#         for i, image_tensor in enumerate(images):
#             print(f"Shape of image {i}: {image_tensor.shape}")
        
#         print("Target label size:", len(target_label))
        
        if it % 10 == 0:
            print(f"이터레이션: {it}")
        it = len(data_loader) * epoch + it  # global training iteration
        for i, param_group in enumerate(optimizer.param_groups):
            param_group["lr"] = lr_schedule[it]
            if i == 0:  # only the first group is regularized
                param_group["weight_decay"] = wd_schedule[it]
                
        images = [im.cuda(non_blocking=True) for im in images]
#         targets = [tl.cuda(non_blocking=True) for tl in target_label]
        
#         print(len(images))
#         print(len(targets))
        targets = torch.tensor(targets).to(device)
        
        ##muti_view_crop 때문에```
#                 Number of images in the list: 10
#         Shape of image 0: torch.Size([64, 3, 224, 224])
#         Shape of image 1: torch.Size([64, 3, 224, 224])
#         Shape of image 2: torch.Size([64, 3, 96, 96])
#         Shape of image 3: torch.Size([64, 3, 96, 96])
#         Shape of image 4: torch.Size([64, 3, 96, 96])
#         Shape of image 5: torch.Size([64, 3, 96, 96])
#         Shape of image 6: torch.Size([64, 3, 96, 96])
#         Shape of image 7: torch.Size([64, 3, 96, 96])
#         Shape of image 8: torch.Size([64, 3, 96, 96])
#         Shape of image 9: torch.Size([64, 3, 96, 96])
#         Target label size: 64
#         이런구조임
        
         
        with torch.cuda.amp.autocast(fp16_scaler is not None):
            teacher_output = teacher(images[:2])  # only the 2 global views pass through the teacher
            student_output = student(images)
            loss = dino_loss(student_output, teacher_output, epoch, targets)
            print("loss : ", loss)
            
        # student update
        optimizer.zero_grad()
        param_norms = None
        if fp16_scaler is None:
            loss.backward()
            if args.clip_grad:
                param_norms = utils.clip_gradients(student, args.clip_grad)
#             utils.cancel_gradients_last_layer(epoch, student,
#                                               args.freeze_last_layer)
            optimizer.step()
    

    
        with torch.no_grad():
            m = momentum_schedule[it]  # momentum parameter
            #teacher의 가중치를 student에게 가르쳐줌
            for param_q, param_k in zip(student.parameters(), teacher_without_ddp.parameters()):
                param_k.data.mul_(m).add_((1 - m) * param_q.detach().data)
                
    # ============ writing logs ... ============
    save_dict = {
        'student': student.state_dict(),
        'teacher': teacher.state_dict(),
        'optimizer': optimizer.state_dict(),
        'epoch': epoch + 1,
        'args': args,
        'dino_loss': dino_loss.state_dict(),
    }
    
total_time = time.time() - start_time
total_time_str = str(datetime.timedelta(seconds=int(total_time)))
print('Training time {}'.format(total_time_str))





Starting DINO training !


  0%|                                                        | 0/100 [00:00<?, ?it/s]
0it [00:00, ?it/s]/tmp/ipykernel_104050/430633712.py:24: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  targets = torch.tensor(targets).to(device)

1it [00:01,  1.68s/it]

이터레이션: 0
predicted_labels : tensor([57,  8, 61,  8, 62,  8, 47,  8,  8, 62,  8,  8, 57, 42, 62, 15, 72, 98,
        62, 42,  8, 87,  7, 57, 57, 19,  8, 36,  7, 70, 62, 70,  8, 15, 87,  8,
        37,  8,  8,  8,  6,  8,  8, 39, 61,  8,  8, 44,  8, 62,  8, 72, 57,  8,
         8, 96, 87,  8, 87,  8, 71,  8,  8, 38], device='cuda:0')
accuracy : 0.0
loss :  (tensor(4.8090, device='cuda:0', grad_fn=<DivBackward0>), 0.0)



2it [00:03,  1.65s/it]

predicted_labels : tensor([96,  6,  5, 61, 38, 96, 93, 25, 93, 71, 49, 52,  8, 42, 87, 62, 62, 62,
        62, 87, 99, 28,  8, 86,  8, 42, 44, 62,  8,  8,  8, 30,  7,  6, 42, 61,
        87, 78, 52, 57,  8, 87, 15, 71, 25, 44, 62, 83,  5,  5, 52, 87, 84, 42,
        62, 61, 84,  8,  7,  5,  8, 52, 78,  5], device='cuda:0')
accuracy : 0.015625
loss :  (tensor(4.7084, device='cuda:0', grad_fn=<DivBackward0>), 0.015625)


...

predicted_labels : tensor([61, 93, 70, 96, 84, 84, 96, 96, 27, 62, 85,  5, 15, 49,  8, 72, 42,  7,
        97,  6, 84, 52, 70, 84, 91,  8,  8,  8,  8, 87, 27, 38,  7, 84, 19, 38,
        52, 93, 15, 33, 62,  8,  8, 90, 87, 61, 39, 52, 84,  3,  1,  8, 42, 62,
        62,  1, 62,  8,  8, 87,  8, 52, 15, 62], device='cuda:0')
accuracy : 0.015625
loss :  (tensor(4.7751, device='cuda:0', grad_fn=<DivBackward0>), 0.015625)



28it [00:40,  1.33s/it]

predicted_labels : tensor([57, 52, 52, 87,  8,  8,  8,  8, 76, 87,  8, 38, 93, 52,  8, 44, 52, 44,
        84, 47,  8, 85,  8, 45, 71,  8, 96, 52,  7,  8, 96,  8, 93, 62,  6, 44,
        52, 24, 76, 61, 91,  8, 87,  8,  4, 59, 49,  8, 38,  8, 57, 62, 30, 37,
        90, 61,  7, 90,  8, 62,  8, 44, 38, 87], device='cuda:0')
accuracy : 0.015625
loss :  (tensor(4.8214, device='cuda:0', grad_fn=<DivBackward0>), 0.015625)


28it [00:40,  1.46s/it]
  0%|                                                        | 0/100 [00:40<?, ?it/s]


KeyboardInterrupt: 